<a href="https://colab.research.google.com/github/ufrpe-ensino/ic-aulas/blob/master/aulas/15_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dask

Adaptado deste [tutorial](https://colab.research.google.com/github/dask/dask-examples/blob/main/dataframes/01-data-access.ipynb#scrollTo=gAg2OgAbX-0h)

Como conjuntos de dados e cálculos são escalonados mais rápido do que CPUs e RAM, precisamos encontrar maneiras de escalar nossos cálculos em várias máquinas.

O [Dask](https://docs.dask.org/en/latest/) fornece maneiras de dimensionar os fluxos de trabalho do Pandas, Scikit-Learn e Numpy de forma mais nativa, com o mínimo de reescrita. Ele se integra bem com essas ferramentas para que copie a maior parte de sua API e use suas estruturas de dados internamente.

## Dask vs Spark

O Dask é menor e mais leve do que o Spark. Isso significa que ele tem menos recursos e, em vez disso, é usado em conjunto com outras bibliotecas, especialmente aquelas no ecossistema Python numérico. 

O framework é escrito em Python e só realmente suporta Python (ao contrário de Spark, cuja base é feita em Scala). Ele interopera bem com C / C ++ / Fortran / LLVM ou outro código compilado nativamente vinculado por meio de Python.


## Instalando

In [4]:
!pip install "dask[complete]"    # Install everything
!pip install dask distributed --upgrade

     |████████████████████████████████| 942kB 8.4MB/s 
Requirement already up-to-date: distributed in /usr/local/lib/python3.7/dist-packages (2021.4.0)
ERROR: distributed 2021.4.0 has requirement cloudpickle>=1.5.0, but you'll have cloudpickle 1.3.0 which is incompatible.
  Found existing installation: dask 2.12.0
    Uninstalling dask-2.12.0:
      Successfully uninstalled dask-2.12.0


## Dask Client

Iniciar o Dask Client é opcional. Ele fornecerá um painel útil para obter informações sobre a computação.

O link para o painel ficará visível quando você criar o cliente abaixo. Recomendamos que ele seja aberto em uma aba a parte

In [1]:
# Obs: não acessível via google colab
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, 
                processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://172.28.0.2/155/1 Dashboard: http://172.28.0.2:8787/status,Cluster Workers: 1 Cores: 4 Memory: 1.86 GiB


## Criar dataset sintético

In [3]:
import os
import datetime
import dask

df = dask.datasets.timeseries()

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,990,Ursula,0.5910619014550973,0.6097891416553023
2000-01-01 00:00:01,950,Laura,-0.6548899301762361,0.3330548746055497
2000-01-01 00:00:02,973,Quinn,0.787237333782858,0.18027359070270021
2000-01-01 00:00:03,1017,Patricia,0.4346032412735221,-0.530045323987619
2000-01-01 00:00:04,966,Victor,-0.9114323316534967,-0.831820055154882
2000-01-01 00:00:05,990,Norbert,0.5157563278166954,0.7479333268532384
2000-01-01 00:00:06,973,Michael,0.7796822828835464,-0.18818257457778387
2000-01-01 00:00:07,899,George,0.3549433004401792,0.29407511362212735
2000-01-01 00:00:08,994,Wendy,0.4779116364950582,0.3577422015799867


## Lendo múltiplos arquivos em um único dataframe

In [7]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,990,Ursula,0.591062,0.609789
1,2000-01-01 00:00:01,950,Laura,-0.654890,0.333055
2,2000-01-01 00:00:02,973,Quinn,0.787237,0.180274
3,2000-01-01 00:00:03,1017,Patricia,0.434603,-0.530045
4,2000-01-01 00:00:04,966,Victor,-0.911432,-0.831820
